In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings
from math import exp
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Ignore warning messages.
warnings.filterwarnings("ignore") 

# Read data from the csv file and convert it into a dataframe.
df = pd.read_csv('/Users/andrewzhou/Documents/2023 Summer Research/Organized-noavg.csv')

df.head()

,Coordinates and Times,.ALPHA.-ENDOSULFAN,.ALPHA.-HEXACHLOROCYCLOHEXANE,.BETA.-ENDOSULFAN,.BETA.-HEXACHLOROCYCLOHEXANE,.DELTA.-HEXACHLOROCYCLOHEXANE,.LAMBDA.-CYHALOTHRIN,"1-(3,4-DICHLOROPHENYL)-3-METHYL UREA",1-CHLOROPROPANE,"1,1-DICHLOROETHANE",...,WIND CONDITION (CHOICE LIST),"WIND DIRECTION (DIRECTION FROM, EXPRESSED 0-360 DEG)",WIND DIRECTION FROM (CHOICE LIST),WIND SPEED (CHOICE LIST),WIND VELOCITY,XYLENE,YTTERBIUM,YTTRIUM,ZINC,ZOXAMIDE
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8/30/22 14:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(blank),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18.41459765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#look at the tail of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69031 entries, 0 to 69030
Columns: 714 entries, Coordinates and Times to ZOXAMIDE
dtypes: float64(713), object(1)
memory usage: 376.0+ MB


In [22]:
df.drop(columns='Coordinates and Times',axis=1,inplace=True)
df.drop(columns='ORTHOPHOSPHATE',axis=1,inplace=True)


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69031 entries, 0 to 69030
Columns: 712 entries, .ALPHA.-ENDOSULFAN to ZOXAMIDE
dtypes: float64(712)
memory usage: 375.0 MB


In [23]:
#df['NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)'] = pd.to_numeric(df['NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)'], errors='coerce')
#df = df.replace(np.nan, 0, regex=True)
#df.info

In [24]:
#df['CHLOROPHYLL A, UNCORRECTED FOR PHEOPHYTIN'] = pd.to_numeric(df['CHLOROPHYLL A, UNCORRECTED FOR PHEOPHYTIN'], errors='coerce')
#df = df.replace(np.nan, 0, regex=True)
#df.info

In [25]:
#df=df.astype('float')
#df = df.replace(np.nan,0,regex=True)
df.dtypes

.ALPHA.-ENDOSULFAN               float64
.ALPHA.-HEXACHLOROCYCLOHEXANE    float64
.BETA.-ENDOSULFAN                float64
.BETA.-HEXACHLOROCYCLOHEXANE     float64
.DELTA.-HEXACHLOROCYCLOHEXANE    float64
                                  ...   
XYLENE                           float64
YTTERBIUM                        float64
YTTRIUM                          float64
ZINC                             float64
ZOXAMIDE                         float64
Length: 712, dtype: object

In [26]:
#df.plot(kind = 'scatter', y = 'NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)', x = 'CHLOROPHYLL A, UNCORRECTED FOR PHEOPHYTIN',ylim=(0,4000),xlim=(0,700))
#plt.show()

In [27]:
#Splitting into train and test sets
X=df.loc[:,df.columns != 'PHOSPHORUS']     
X.info()
Y=df['PHOSPHORUS']  
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state = 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69031 entries, 0 to 69030
Columns: 711 entries, .ALPHA.-ENDOSULFAN to ZOXAMIDE
dtypes: float64(711)
memory usage: 374.5 MB


In [28]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

In [29]:
from sklearn.linear_model import LinearRegression


model = LinearRegression()
model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

model = RandomForestRegressor()


model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')



In [ ]:
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
important_features = pd.Series(data=model.feature_importances_,index=X_train.columns)
important_features.sort_values(ascending=False,inplace=True)
print(important_features)

New dataframe with only important features 

In [ ]:
sdf = pd.DataFrame()
sdf['NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)'] = df['NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)']
sdf['PHOSPHORUS'] = df['PHOSPHORUS']
sdf['CHLOROPHYLL A, UNCORRECTED FOR PHEOPHYTIN'] = df['CHLOROPHYLL A, UNCORRECTED FOR PHEOPHYTIN']
sdf['DEPTH, SECCHI DISK DEPTH'] = df['DEPTH, SECCHI DISK DEPTH']
sdf.info()

In [ ]:
#Making train and test sets only from important features
X=sdf.loc[:,sdf.columns != 'NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)']     
X.info()
Y=sdf['NITROGEN, MIXED FORMS (NH3), (NH4), ORGANIC, (NO2) AND (NO3)']  
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state = 1)

In [ ]:
#Model Accuracy with only 3 most important features
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

model = RandomForestRegressor()


model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


model = KNeighborsRegressor()


model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')

In [ ]:


from sklearn.tree import DecisionTreeRegressor


model = DecisionTreeRegressor()


model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')

In [ ]:
from sklearn import svm


model = svm.SVR()
model.fit(X_train, Y_train)
train_accuracy = model.score(X_train, Y_train)
print(f'The accuracy for the training set is {100 * train_accuracy:.2f}%')
test_accuracy = model.score(X_test, Y_test)
print(f'The accuracy for the test set is {100 * test_accuracy:.2f}%')